In [ ]:
# 成品

In [1]:
import sys
sys.path.append('C:/Users/sheng/Desktop/F_Market/Trading')
import utility_f as uf
import pandas as pd
import yfinance as yf
import numpy as np
import datetime
import time
import traceback
try:
    #讀取股票清單
    data = pd.read_excel('C:/Users/sheng/Desktop/F_Market/Trading/stock_list.xlsx')
    #獲取代號
    target_stock = data['代號'].tolist()
    #獲取今日日期
    today = datetime.date.today()
    #傳入判斷今天是否為營業日
    if_trade = uf.is_open(today)
    #如果是N，表示沒開盤，準備寄信
    if if_trade=='N':
        #沒開盤應該收件者
        mail_list = ['a*****@gmail.com']
        #標題為三大法人篩選，今日休市
        subject = f'{today} 小幫手暴漲中的股票偵測 - 今日休市'
        #郵件內容為空
        body = ''
        #寄信
        uf.send_mail(mail_list, subject, body, 'text',None, None)
        exit()
    #獲取過去10天的收盤價
    date_start = today + datetime.timedelta(days =-20)
    #轉為str格式等一下準備傳入yfinance的history獲取歷史股價
    date_start = date_start.strftime('%Y-%m-%d')
    #獲取T+1日，因為yfinance的end日期是到T-1
    date_end = today + datetime.timedelta(days =1)
    #轉為str格式等一下準備傳入yfinance的history獲取歷史股價
    date_end = date_end.strftime('%Y-%m-%d')
    #創建空list備用
    stock_store = []
    today_store = []
    today_fall = []
    count = 0
    #迴圈處理每一支目標股票
    for target in target_stock:
        count+=1
        time.sleep(1)
        #獲取目標股票的Ticker類
        stock = yf.Ticker(f'{target}.TW')
        #傳入start跟end獲取歷史股價
        df = stock.history(start=date_start,end=date_end) 
        #做一點基本的檢核，防止資料回傳0筆或1筆
        if len(df) >=5:
            #將收盤轉為array並且取最後一筆，今天的收盤
            today_price = df['Close'].values[-1]
            #將收盤轉為array並且取倒數第二筆，昨天的收盤
            yes_price = df['Close'].values[-2]
            #將收盤轉為array並且取倒數第二筆，前天的收盤
            be_yes_price = df['Close'].values[-3]
            #根據公式取得今日的漲跌幅
            fall_today = ((today_price - yes_price)/yes_price)*100
            #根據公式取得昨日的漲跌幅
            fall_yes = ((yes_price-be_yes_price)/be_yes_price)*100
            #兩個漲跌幅皆大於5則是我們的目標，儲存起來。
            if fall_today>=5 and fall_yes>=5:
                print(f'Stock: { target} | fall today: {fall_today} | today: {today_price} | yes: {yes_price}')
                today_store.append(today_price)
                today_fall.append(fall_today)
                stock_store.append(target)
            #print出處理進度
            print(f'Dealing Stock: {target} | All Stock: {len(target_stock)} | Now: {count}')
    #控制用
    print('Get it:',stock_store)
    main_df = pd.DataFrame()
    #Loop剛剛篩選的結果
    for t in stock_store:
        #使用到跟requests有關的東西，習慣先sleep一下
        time.sleep(1)
        #我們要將目標股票的新聞連接再一起，contorl=0代表第一次loop
        #使用新聞函數獲取新聞，並且先獲取主要的dataframe，其他股票的就連接在後面
        #merge_df = uf.get_yahoo_news(t,1)
        merge_df = uf.get_yahoo_news2(t,1)
        #多一個欄位叫做stock儲存這篇新聞是屬於那一支股票
        merge_df['stock'] = len(merge_df)*[t]
        main_df = main_df.append(merge_df)
    #處理要寄信的部分，首先將剛剛的新聞列表儲存成excel保存
    main_df.to_excel(f'C:/Users/sheng/Desktop/F_Market/Trading/fall_stock_news.xlsx',index=False)
    #處理要寄信的部分，這裡處理我希望放在信件body的暴跌中的股票清單
    empty_dataframe = pd.DataFrame()
    empty_dataframe['股票代號'] = stock_store
    empty_dataframe['今價'] = today_store
    empty_dataframe['今日漲幅%'] = today_fall
    #希望寄出的是表格，因此我們將他轉為html備用
    empty_dataframe = empty_dataframe.to_html(index=False)
    #製作html格式
    body = f'''<html>
                <font face="微軟正黑體"></font>
                <body>
                <h4>
                小幫手系列偵測下表股票為暴漲中股票 
                </h4>
                {empty_dataframe}
                <h5>投資理財有賺有賠，請謹慎評估風險</h5>
                </body>
                </html>'''
    #寄信名單
    mail_list = ['a******@gmail.com','0******@gm.scu.edu.tw']
    #標題我們加上日期，淺顯易懂
    subject = f'{today}  小幫手暴漲中股票偵測'
    #寄信
    uf.send_mail(mail_list, subject, body,'html', [f'C:/Users/sheng/Desktop/F_Market/Trading/fall_stock_news.xlsx'], ['相關新聞表.xlsx'])
except SystemExit:
    print('Its OK')
except:
    today = datetime.date.today()
    #收件名單
    mail_list = ['a******@gmail.com','0******@gm.scu.edu.tw']
    #標題我們加上日期，淺顯易懂
    subject = f'{today}  小幫手暴漲中股票異常'
    #內容就是剛剛篩選完成的股票
    body = traceback.format_exc()
    #寄信
    uf.send_mail(mail_list, subject, body,'text', None, None)

Dealing Stock: 1101 | All Stock: 979 | Now: 1
Dealing Stock: 1102 | All Stock: 979 | Now: 2
Dealing Stock: 1103 | All Stock: 979 | Now: 3
Dealing Stock: 1104 | All Stock: 979 | Now: 4
Dealing Stock: 1108 | All Stock: 979 | Now: 5
Dealing Stock: 1109 | All Stock: 979 | Now: 6
Dealing Stock: 1110 | All Stock: 979 | Now: 7
Dealing Stock: 1201 | All Stock: 979 | Now: 8
Dealing Stock: 1203 | All Stock: 979 | Now: 9
Dealing Stock: 1210 | All Stock: 979 | Now: 10
Dealing Stock: 1213 | All Stock: 979 | Now: 11
Dealing Stock: 1215 | All Stock: 979 | Now: 12
Dealing Stock: 1216 | All Stock: 979 | Now: 13
Dealing Stock: 1217 | All Stock: 979 | Now: 14
Dealing Stock: 1218 | All Stock: 979 | Now: 15
Dealing Stock: 1219 | All Stock: 979 | Now: 16
Dealing Stock: 1220 | All Stock: 979 | Now: 17
Dealing Stock: 1225 | All Stock: 979 | Now: 18
Dealing Stock: 1227 | All Stock: 979 | Now: 19
Dealing Stock: 1229 | All Stock: 979 | Now: 20
Dealing Stock: 1231 | All Stock: 979 | Now: 21
Dealing Stock: 1232 | 

Dealing Stock: 1727 | All Stock: 979 | Now: 172
Dealing Stock: 1730 | All Stock: 979 | Now: 173
Dealing Stock: 1731 | All Stock: 979 | Now: 174
Dealing Stock: 1732 | All Stock: 979 | Now: 175
Dealing Stock: 1733 | All Stock: 979 | Now: 176
Dealing Stock: 1734 | All Stock: 979 | Now: 177
Dealing Stock: 1735 | All Stock: 979 | Now: 178
Dealing Stock: 1736 | All Stock: 979 | Now: 179
Dealing Stock: 1737 | All Stock: 979 | Now: 180
Dealing Stock: 1752 | All Stock: 979 | Now: 181
Dealing Stock: 1760 | All Stock: 979 | Now: 182
Dealing Stock: 1762 | All Stock: 979 | Now: 183
Dealing Stock: 1773 | All Stock: 979 | Now: 184
Dealing Stock: 1776 | All Stock: 979 | Now: 185
Dealing Stock: 1783 | All Stock: 979 | Now: 186
Dealing Stock: 1786 | All Stock: 979 | Now: 187
Dealing Stock: 1789 | All Stock: 979 | Now: 188
Dealing Stock: 1795 | All Stock: 979 | Now: 189
Dealing Stock: 1802 | All Stock: 979 | Now: 190
Dealing Stock: 1805 | All Stock: 979 | Now: 191
Dealing Stock: 1806 | All Stock: 979 | N

Dealing Stock: 2426 | All Stock: 979 | Now: 341
Dealing Stock: 2427 | All Stock: 979 | Now: 342
Dealing Stock: 2428 | All Stock: 979 | Now: 343
Dealing Stock: 2429 | All Stock: 979 | Now: 344
Dealing Stock: 2430 | All Stock: 979 | Now: 345
Dealing Stock: 2431 | All Stock: 979 | Now: 346
Dealing Stock: 2433 | All Stock: 979 | Now: 347
Dealing Stock: 2434 | All Stock: 979 | Now: 348
Dealing Stock: 2436 | All Stock: 979 | Now: 349
Dealing Stock: 2438 | All Stock: 979 | Now: 350
Dealing Stock: 2439 | All Stock: 979 | Now: 351
Dealing Stock: 2440 | All Stock: 979 | Now: 352
Dealing Stock: 2441 | All Stock: 979 | Now: 353
Dealing Stock: 2442 | All Stock: 979 | Now: 354
Dealing Stock: 2443 | All Stock: 979 | Now: 355
Dealing Stock: 2444 | All Stock: 979 | Now: 356
Dealing Stock: 2449 | All Stock: 979 | Now: 357
Dealing Stock: 2450 | All Stock: 979 | Now: 358
Dealing Stock: 2451 | All Stock: 979 | Now: 359
Dealing Stock: 2453 | All Stock: 979 | Now: 360
Dealing Stock: 2454 | All Stock: 979 | N

Dealing Stock: 3010 | All Stock: 979 | Now: 512
Dealing Stock: 3011 | All Stock: 979 | Now: 513
Dealing Stock: 3013 | All Stock: 979 | Now: 514
Dealing Stock: 3014 | All Stock: 979 | Now: 515
Dealing Stock: 3015 | All Stock: 979 | Now: 516
Dealing Stock: 3016 | All Stock: 979 | Now: 517
Dealing Stock: 3017 | All Stock: 979 | Now: 518
Dealing Stock: 3018 | All Stock: 979 | Now: 519
Dealing Stock: 3019 | All Stock: 979 | Now: 520
Dealing Stock: 3021 | All Stock: 979 | Now: 521
Dealing Stock: 3022 | All Stock: 979 | Now: 522
Dealing Stock: 3023 | All Stock: 979 | Now: 523
Dealing Stock: 3024 | All Stock: 979 | Now: 524
Dealing Stock: 3025 | All Stock: 979 | Now: 525
Dealing Stock: 3026 | All Stock: 979 | Now: 526
Dealing Stock: 3027 | All Stock: 979 | Now: 527
Dealing Stock: 3028 | All Stock: 979 | Now: 528
Dealing Stock: 3029 | All Stock: 979 | Now: 529
Dealing Stock: 3030 | All Stock: 979 | Now: 530
Dealing Stock: 3031 | All Stock: 979 | Now: 531
Dealing Stock: 3032 | All Stock: 979 | N

Dealing Stock: 4746 | All Stock: 979 | Now: 683
Dealing Stock: 4755 | All Stock: 979 | Now: 684
Dealing Stock: 4763 | All Stock: 979 | Now: 685
Dealing Stock: 4764 | All Stock: 979 | Now: 686
Dealing Stock: 4766 | All Stock: 979 | Now: 687
Dealing Stock: 4770 | All Stock: 979 | Now: 688
Dealing Stock: 4807 | All Stock: 979 | Now: 689
Dealing Stock: 4904 | All Stock: 979 | Now: 690
Dealing Stock: 4906 | All Stock: 979 | Now: 691
Dealing Stock: 4912 | All Stock: 979 | Now: 692
Dealing Stock: 4915 | All Stock: 979 | Now: 693
Dealing Stock: 4916 | All Stock: 979 | Now: 694
Dealing Stock: 4919 | All Stock: 979 | Now: 695
Dealing Stock: 4927 | All Stock: 979 | Now: 696
Dealing Stock: 4930 | All Stock: 979 | Now: 697
Dealing Stock: 4934 | All Stock: 979 | Now: 698
Dealing Stock: 4935 | All Stock: 979 | Now: 699
Dealing Stock: 4938 | All Stock: 979 | Now: 700
Dealing Stock: 4942 | All Stock: 979 | Now: 701
Dealing Stock: 4943 | All Stock: 979 | Now: 702
Dealing Stock: 4952 | All Stock: 979 | N

Dealing Stock: 6671 | All Stock: 979 | Now: 854
Dealing Stock: 6672 | All Stock: 979 | Now: 855
Dealing Stock: 6674 | All Stock: 979 | Now: 856
Dealing Stock: 6689 | All Stock: 979 | Now: 857
Dealing Stock: 6691 | All Stock: 979 | Now: 858
Dealing Stock: 6695 | All Stock: 979 | Now: 859
Dealing Stock: 6698 | All Stock: 979 | Now: 860
Dealing Stock: 6706 | All Stock: 979 | Now: 861
Dealing Stock: 6715 | All Stock: 979 | Now: 862
Dealing Stock: 6719 | All Stock: 979 | Now: 863
Dealing Stock: 6743 | All Stock: 979 | Now: 864
Dealing Stock: 6753 | All Stock: 979 | Now: 865
Dealing Stock: 6754 | All Stock: 979 | Now: 866
Dealing Stock: 6756 | All Stock: 979 | Now: 867
Dealing Stock: 6768 | All Stock: 979 | Now: 868
Dealing Stock: 6770 | All Stock: 979 | Now: 869
Dealing Stock: 6776 | All Stock: 979 | Now: 870
Dealing Stock: 6781 | All Stock: 979 | Now: 871
Dealing Stock: 6782 | All Stock: 979 | Now: 872
Dealing Stock: 6789 | All Stock: 979 | Now: 873
Dealing Stock: 6790 | All Stock: 979 | N

C:\Users\sheng\AppData\Local\Temp\ipykernel_15080\2660357560.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_df = main_df.append(merge_df)
C:\Users\sheng\AppData\Local\Temp\ipykernel_15080\2660357560.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_df = main_df.append(merge_df)


In [ ]:
# 測試

In [ ]:
import sys
sys.path.append('C:/Users/sheng/Desktop/F_Market/Trading')
import utility_f as uf
import pandas as pd
import yfinance as yf
import numpy as np
import datetime
import time
import traceback

#讀取股票清單
data = pd.read_excel('C:/Users/sheng/Desktop/F_Market/Trading/stock_list.xlsx')
#獲取代號
target_stock = data['代號'].tolist()
#獲取今日日期
today = datetime.date.today()
#傳入判斷今天是否為營業日
if_trade = uf.is_open(today)
#如果是N，表示沒開盤，準備寄信
if if_trade=='N':
    #沒開盤應該收件者
    #mail_list = ['a*****@gmail.com']
    mail_list = ['xx@gmail.com','xxx@gmail.com']
    #標題為三大法人篩選，今日休市
    subject = f'{today} 小幫手暴漲中的股票偵測 - 今日休市'
    #郵件內容為空
    body = ''
    #寄信
    uf.send_mail(mail_list, subject, body, 'text',None, None)
    exit()

 #獲取過去10天的收盤價
date_start = today + datetime.timedelta(days =-20)
#轉為str格式等一下準備傳入yfinance的history獲取歷史股價
date_start = date_start.strftime('%Y-%m-%d')
#獲取T+1日，因為yfinance的end日期是到T-1
date_end = today + datetime.timedelta(days =1)
#轉為str格式等一下準備傳入yfinance的history獲取歷史股價
date_end = date_end.strftime('%Y-%m-%d')

#創建空list備用
stock_store = []
today_store = []
today_fall = []
count = 0
#迴圈處理每一支目標股票
for target in target_stock:
    count+=1
    time.sleep(1)
    #獲取目標股票的Ticker類
    stock = yf.Ticker(f'{target}.TW')
    #傳入start跟end獲取歷史股價
    df = stock.history(start=date_start,end=date_end) 
    #做一點基本的檢核，防止資料回傳0筆或1筆
    if len(df) >=5:
        #將收盤轉為array並且取最後一筆，今天的收盤
        today_price = df['Close'].values[-1]
        #將收盤轉為array並且取倒數第二筆，昨天的收盤
        yes_price = df['Close'].values[-2]
        #將收盤轉為array並且取倒數第二筆，前天的收盤
        be_yes_price = df['Close'].values[-3]
        #根據公式取得今日的漲跌幅
        fall_today = ((today_price - yes_price)/yes_price)*100
        #根據公式取得昨日的漲跌幅
        fall_yes = ((yes_price-be_yes_price)/be_yes_price)*100
        #兩個漲跌幅皆大於5則是我們的目標，儲存起來。
        if fall_today>=5 and fall_yes>=5:
            print(f'Stock: { target} | fall today: {fall_today} | today: {today_price} | yes: {yes_price}')
            today_store.append(today_price)
            today_fall.append(fall_today)
            stock_store.append(target)
        #print出處理進度
        print(f'Dealing Stock: {target} | All Stock: {len(target_stock)} | Now: {count}')

  

In [ ]:
print(stock_store)


In [ ]:
import sys
sys.path.append('C:/Users/sheng/Desktop/F_Market/Trading')
import utility_f as uf
print(uf.get_yahoo_news2(1560))

In [ ]:
def get_yahoo_news2(stock: str, num_news: int):
    # 準備headers
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36"
    }
    # requests指定網址，並且帶入headers
    data = requests.get(
        f"https://tw.stock.yahoo.com/quote/{stock}/news", headers=headers
    )
    # 使用BeautifulSoup解析，指定解析方法html.parser可以避免warning
    soup = BeautifulSoup(data.text, "html.parser")

    # 選取標題的tag來使用，讀者自行練習的時候應該會發現有許多tag可以使用，基本上可以使用就可以，未必要跟我的一樣
    all_news = soup.find_all("h3", {"class": "Mt(0) Mb(8px)"})

    # 創建空list儲存每一篇新聞的連結
    all_news_store = []
    # loop每一個抓回來的元素
    for a in all_news:
        # 藉由find獲得的物件還可以再find，因此我們find a元素，並且取得裡面的href，也就是網址
        news_path = a.find("a")["href"]
        # 雅虎新聞目前的設計是中間會夾雜一兩篇廣告，這些廣告的特點是他們的連結最後沒有html的文字，因此可以用這個方法剔除他們
        if news_path[-4:] == "html":
            # 最後append進list備用
            all_news_store.append(news_path)
    
    # 選取指定数量的新聞
    all_news_store = all_news_store[:num_news]

    # 開始進行獲取日期，標題我也在這裡做，當然如果你要再上一個步驟就獲取標題也可以
    # 創建空list備用
    date_store, title_store = [], []
    # loop剛剛獲得的每一篇新聞網址
    for new in all_news_store:
        # 做requests
        each_data = requests.get(f"{new}", headers=headers)
        # BeautifulSoup整理格式，指定解析方法html.parser可以避免warning
        each_soup = BeautifulSoup(each_data.text, "html.parser")
        # find獲取title
        title = each_soup.find("h1", {"data-test-locator": "headline"}).text
        # find獲取時間
        news_time = each_soup.find("div", {"class": "caas-attr-time-style"}).text
        # 整理時間格式，只想要前面的年月日資訊，中間有空白剛好可以利用split切割空白，取第一個元素
        news_time = news_time.split(" ")[0]
        # 這裡看個人要不要做，我不喜歡例如2021年9月27日這樣的格式，因此我用replace轉換成2021/9/27
        news_time = news_time.replace("年", "/")
        news_time = news_time.replace("月", "/")
        news_time = news_time.replace("日", "")
        # append整理資料
        title_store.append(title)
        date_store.append(news_time)
    # 整理成DataFrame
    result = pd.DataFrame()
    result["title"] = title_store
    result["url"] = all_news_store
    result["date"] = date_store
    return result

In [ ]:
import requests
from bs4 import BeautifulSoup
print(get_yahoo_news2(2301,2))

In [ ]:
stock_store = [1560, 2301, 2356, 2388, 2419, 3037, 3231, 3443, 3518, 3617, 3706, 3715, 5607, 6442, 6515, 6579, 8201]

In [ ]:
import sys
sys.path.append('C:/Users/sheng/Desktop/F_Market/Trading')
import utility_f as uf
import pandas as pd
import yfinance as yf
import numpy as np
import datetime
import time
import traceback

main_df = pd.DataFrame()
#Loop剛剛篩選的結果
for t in stock_store:
    #使用到跟requests有關的東西，習慣先sleep一下
    time.sleep(1)
    #我們要將目標股票的新聞連接再一起，contorl=0代表第一次loop
    #使用新聞函數獲取新聞，並且先獲取主要的dataframe，其他股票的就連接在後面
    #merge_df = uf.get_yahoo_news(t,1)
    merge_df = uf.get_yahoo_news2(t,2)
    #多一個欄位叫做stock儲存這篇新聞是屬於那一支股票
    merge_df['stock'] = len(merge_df)*[t]
    main_df = main_df.append(merge_df)



In [ ]:
print(main_df)

In [ ]:
#處理要寄信的部分，首先將剛剛的新聞列表儲存成excel保存
main_df.to_excel(f'C:/Users/sheng/Desktop/F_Market/Trading/fall_stock_news.xlsx',index=False)